In [26]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [27]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [28]:
tf.keras.backend.clear_session()

In [29]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [30]:
subject = 'Rio De Janeiro - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [31]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_RJ.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Ano,Rio de Janeiro - Desemprego,Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH Educacao,Rio de Janeiro - IDH,Rio de Janeiro - IDH Longevidade,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,...,Taxa Selic (%),Rio de Janeiro - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio De Janeiro - Consumo de Cimento (t)
0,2008,8.106163,3010.012911,0.740781,0.777194,0.819163,3.769087e+08,1.848763e+07,22.305022,3.507645e+08,...,11.760983,0.773683,36.697984,1012.203296,1103.619545,23.145598,19.041313,80.733596,0.819193,4300.676
1,2009,8.134661,3108.577595,0.743129,0.778002,0.820366,3.784121e+08,1.857076e+07,22.275112,3.502037e+08,...,9.472585,0.772684,32.933440,1012.149927,1113.135070,23.101387,19.160541,81.195790,0.873595,3966.930
2,2010,8.098147,3181.826392,0.745612,0.778838,0.820996,3.868458e+08,1.857479e+07,22.286297,3.512210e+08,...,9.332485,0.772158,29.177236,1012.427323,1141.349974,22.850128,18.952421,81.281067,0.913459,4199.911
3,2011,8.084479,3209.638424,0.748230,0.779704,0.821062,3.942901e+08,1.848968e+07,22.267033,3.516849e+08,...,10.985689,0.772096,25.153610,1012.733462,1164.724634,22.787473,18.939670,81.492721,0.874916,4390.254
4,2012,9.891336,3199.169780,0.750390,0.780264,0.820467,4.002860e+08,1.838631e+07,22.204155,3.513894e+08,...,8.159014,0.772259,23.695979,1012.912153,1180.075995,22.961436,18.924341,80.811760,0.826735,4733.632
5,2013,10.961499,3078.657877,0.754339,0.781786,0.819506,4.047278e+08,1.820862e+07,22.108996,3.504515e+08,...,7.913561,0.773613,22.343427,1012.853635,1205.657982,23.167187,18.786695,79.473074,0.807596,4864.477
6,2014,11.800434,2795.256292,0.757618,0.782935,0.818048,4.076404e+08,1.791300e+07,21.995673,3.490706e+08,...,10.350442,0.774998,22.049108,1012.787954,1202.470602,23.235371,18.867735,79.539625,0.790897,4454.468
7,2015,12.471375,2440.539373,0.759419,0.783180,0.815969,4.088358e+08,1.760007e+07,21.869610,3.473618e+08,...,12.454398,0.775914,23.113331,1013.001425,1202.552964,22.947408,18.812497,80.526411,0.779173,3639.072
8,2016,12.865739,2231.595625,0.763524,0.784651,0.823363,4.103079e+08,1.740778e+07,21.823613,3.468569e+08,...,13.111908,0.769071,23.956982,1013.167500,1208.448832,22.622476,18.570007,80.750667,0.763191,2811.648
9,2017,12.728617,2351.836251,0.764117,0.784256,0.820668,4.118381e+08,1.729266e+07,21.853351,3.474898e+08,...,9.479812,0.769805,24.002032,1013.186401,1206.381467,22.689505,18.282287,78.796401,0.738610,2284.918


In [32]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Rio de Janeiro - Desemprego,Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH Educacao,Rio de Janeiro - IDH,Rio de Janeiro - IDH Longevidade,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rio de Janeiro - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,8.106163,3010.012911,0.740781,0.777194,0.819163,3.769087e+08,1.848763e+07,22.305022,3.507645e+08,5.483335e+08,...,2.868108,11.760983,0.773683,36.697984,1012.203296,1103.619545,23.145598,19.041313,80.733596,0.819193
1,8.134661,3108.577595,0.743129,0.778002,0.820366,3.784121e+08,1.857076e+07,22.275112,3.502037e+08,5.302764e+08,...,2.305885,9.472585,0.772684,32.933440,1012.149927,1113.135070,23.101387,19.160541,81.195790,0.873595
2,8.098147,3181.826392,0.745612,0.778838,0.820996,3.868458e+08,1.857479e+07,22.286297,3.512210e+08,5.046425e+08,...,1.958246,9.332485,0.772158,29.177236,1012.427323,1141.349974,22.850128,18.952421,81.281067,0.913459
3,8.084479,3209.638424,0.748230,0.779704,0.821062,3.942901e+08,1.848968e+07,22.267033,3.516849e+08,4.854622e+08,...,2.079088,10.985689,0.772096,25.153610,1012.733462,1164.724634,22.787473,18.939670,81.492721,0.874916
4,9.891336,3199.169780,0.750390,0.780264,0.820467,4.002860e+08,1.838631e+07,22.204155,3.513894e+08,4.867661e+08,...,2.183282,8.159014,0.772259,23.695979,1012.912153,1180.075995,22.961436,18.924341,80.811760,0.826735
5,10.961499,3078.657877,0.754339,0.781786,0.819506,4.047278e+08,1.820862e+07,22.108996,3.504515e+08,5.225848e+08,...,2.403445,7.913561,0.773613,22.343427,1012.853635,1205.657982,23.167187,18.786695,79.473074,0.807596
6,11.800434,2795.256292,0.757618,0.782935,0.818048,4.076404e+08,1.791300e+07,21.995673,3.490706e+08,5.676413e+08,...,2.748397,10.350442,0.774998,22.049108,1012.787954,1202.470602,23.235371,18.867735,79.539625,0.790897
7,12.471375,2440.539373,0.759419,0.783180,0.815969,4.088358e+08,1.760007e+07,21.869610,3.473618e+08,5.179392e+08,...,2.501529,12.454398,0.775914,23.113331,1013.001425,1202.552964,22.947408,18.812497,80.526411,0.779173
8,12.865739,2231.595625,0.763524,0.784651,0.823363,4.103079e+08,1.740778e+07,21.823613,3.468569e+08,4.524144e+08,...,2.073464,13.111908,0.769071,23.956982,1013.167500,1208.448832,22.622476,18.570007,80.750667,0.763191
9,12.728617,2351.836251,0.764117,0.784256,0.820668,4.118381e+08,1.729266e+07,21.853351,3.474898e+08,4.213689e+08,...,2.177386,9.479812,0.769805,24.002032,1013.186401,1206.381467,22.689505,18.282287,78.796401,0.738610


In [33]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     3966.930
1     4199.911
2     4390.254
3     4733.632
4     4864.477
5     4454.468
6     3639.072
7     2811.648
8     2284.918
9     2391.212
10    2715.843
11    2935.607
12    2829.682
13    2682.995
14         NaN
Name: Rio De Janeiro - Consumo de Cimento (t), dtype: float64

In [34]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Rio de Janeiro - Desemprego,Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH Educacao,Rio de Janeiro - IDH,Rio de Janeiro - IDH Longevidade,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rio de Janeiro - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.295032,0.555761,-1.656559,-1.725842,-0.001333,-1.872139,1.005727,1.287005,0.822598,1.204752,...,2.026352,0.984445,0.697873,2.449354,-1.708844,-2.015230,1.154964,0.927567,0.543018,0.299377
1,-1.280741,0.842698,-1.373573,-1.390057,0.459791,-1.751059,1.166146,1.127670,0.493474,0.820657,...,0.085558,-0.051710,0.183065,1.592540,-1.858608,-1.746354,0.930685,1.253839,0.768415,1.160233
2,-1.299051,1.055937,-1.074224,-1.042669,0.700955,-1.071815,1.173922,1.187258,1.090534,0.275392,...,-1.114490,-0.115145,-0.087964,0.737625,-1.080178,-0.949097,-0.343950,0.684314,0.810002,1.791059
3,-1.305905,1.136902,-0.758611,-0.682665,0.726314,-0.472261,1.009679,1.084634,1.362799,-0.132596,...,-0.697344,0.633403,-0.119702,-0.178155,-0.221089,-0.288609,-0.661796,0.649421,0.913219,1.181138
4,-0.399830,1.106426,-0.498118,-0.449756,0.498392,0.010641,0.810215,0.749679,1.189346,-0.104860,...,-0.337667,-0.646476,-0.036164,-0.509913,0.280352,0.145168,0.220713,0.607473,0.581136,0.418710
5,0.136819,0.755597,-0.021996,0.182905,0.130040,0.368375,0.467319,0.242762,0.638882,0.657046,...,0.422337,-0.757614,0.662087,-0.817755,0.116139,0.868027,1.264485,0.230801,-0.071697,0.115857
6,0.557515,-0.069429,0.373240,0.660454,-0.428706,0.602957,-0.103140,-0.360918,-0.171591,1.615453,...,1.613108,0.345772,1.375580,-0.884742,-0.068175,0.777963,1.610382,0.452569,-0.039242,-0.148394
7,0.893969,-1.102065,0.590429,0.762131,-1.225650,0.699234,-0.707000,-1.032469,-1.174519,0.558230,...,0.760923,1.298413,1.847902,-0.642524,0.530867,0.780290,0.149550,0.301409,0.441981,-0.333907
8,1.091729,-1.710333,1.085352,1.373793,1.608208,0.817796,-1.078057,-1.277497,-1.470832,-0.835563,...,-0.716758,1.596124,-1.678745,-0.450508,0.996909,0.946887,-1.498824,-0.362170,0.551343,-0.586816
9,1.022967,-1.360294,1.156858,1.209471,0.575559,0.941033,-1.300219,-1.119081,-1.099399,-1.495939,...,-0.358019,-0.048438,-1.300597,-0.440255,1.049948,0.888471,-1.158787,-1.149523,-0.401688,-0.975792


In [35]:
reshaped_train_input = dfToInputRNN(train_input)

In [36]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.86947128, -0.44639735,  0.9495356 ,  0.21794585,
         -1.3537518 ,  0.75370195, -1.10713693, -0.88220807,
         -0.82325106, -1.0453709 ,  1.21859064, -0.99976018,
          0.79428371,  1.15575749, -0.52559858,  1.30260559,
         -0.55253046, -0.25130359,  1.38787176,  1.47154451,
         -1.80649616,  1.09296291,  0.94281138,  0.52836084,
          0.61258103,  0.87656257,  1.47801536,  1.71982086,
          1.28221181,  1.39086472,  1.58209844,  1.08872091,
          0.55212962, -1.81398118,  1.44113674,  1.11191094,
          1.30470835,  0.86390516, -1.67365968, -1.70738939,
         -1.55813196, -0.48498301, -0.75714639,  0.40770893,
         -0.27394734,  2.02323219,  1.2908951 ,  1.70465513,
          1.81148084,  1.39968653, -1.36191442,  1.28024593,
          1.11054795,  0.12662799,  1.40414112,  1.40468999,
          1.45643959,  1.58509336,  1.32283157,  1.30222047,
          1.55138602,  1.91801602, -0.77489784,  2.32098884,
         -1.1942541 , -1

In [37]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     3966.930
1     4199.911
2     4390.254
3     4733.632
4     4864.477
5     4454.468
6     3639.072
7     2811.648
8     2284.918
9     2391.212
10    2715.843
11    2935.607
Name: Rio De Janeiro - Consumo de Cimento (t), dtype: float64

In [38]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    2935.607
Name: Rio De Janeiro - Consumo de Cimento (t), dtype: float64

In [39]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Rio de Janeiro - Desemprego,Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH Educacao,Rio de Janeiro - IDH,Rio de Janeiro - IDH Longevidade,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rio de Janeiro - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.841176,-0.224005,0.757994,-0.824847,-2.627309,0.3589,-0.879379,-0.909417,-1.121427,1.108405,...,-1.482779,-2.260447,-1.268801,-0.504495,0.492406,-0.267962,-0.64034,-0.480755,-1.410819,-1.69692


In [40]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.84117589, -0.22400502,  0.75799383, -0.8248473 ,
         -2.62730864,  0.35889974, -0.87937882, -0.90941667,
         -1.12142698,  1.10840491,  2.63582309, -0.18995911,
          0.72640489,  2.66833308, -0.032632  ,  0.98802435,
          0.24980494, -0.08955808,  1.85434394,  2.02706568,
         -2.21150703,  1.42938591,  0.85132668,  0.14092386,
          0.30902324,  1.00267904,  2.27970137,  1.11272593,
          0.61007902,  0.89064538,  1.42231438,  1.02650562,
          1.47127593, -1.53201473,  1.73503858,  1.66610613,
          1.0456894 ,  0.55289259, -1.81216593, -2.10342296,
         -2.55377478,  0.41940628, -1.75756445,  0.241023  ,
         -0.98659345,  1.52184916,  1.01398607,  1.49290967,
          1.40764092,  1.87930411, -2.51892072,  2.83356214,
          1.58620924, -0.02524128,  1.54231542,  1.5176119 ,
          1.7355269 ,  1.87941982,  1.54198129,  1.51276712,
          1.83795594,  2.62210693,  1.04993174,  0.96476493,
         -1.48277861, -2

In [41]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    2829.682
Name: Rio De Janeiro - Consumo de Cimento (t), dtype: float64

In [42]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh')),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        batch_size=train_input.shape[0],
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [43]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [45]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2751793689, 2404519939, 3235089947, 3028470271, 485391019, 4283398255, 33677847, 423367431, 936330014, 1034829387, 3547379063, 2100267927, 3062641015, 3902475953, 1530861042, 3508016389, 4174166468, 4201443556, 3768318940, 3508053179, 1461245102, 3750447986, 1504725306, 163016732, 267962083, 1870209102, 2765767677, 3786141251, 158538596, 3066813050, 1387305755, 3609854794, 3576225660, 2777941518, 1032692001, 1625711586, 1493401902, 1688626926, 2239124748, 2603881857, 1908482060, 652950375, 4161755946, 2264618373, 3044321490, 653816613, 1538512943, 1533583633, 3417262686, 3946817132, 3825555592, 4097904520, 319240685, 719559555, 2748263582, 320738168, 2160465764, 3808628121, 979752885, 800636895, 913246650, 3906691778, 1940235957, 1896721448, 3003127101, 395975779, 473329174, 1676164819, 4149347660, 3442666236, 2651364603, 3188710529, 145188028, 3296361462, 1541560693, 2432364906, 566890673, 2332671814, 182238724, 3900760323, 1364633976, 1579915944, 2481777825, 2726952244, 1723049352, 

loss: 23978.556640625


Step: 92 ___________________________________________
loss: 22510.1484375


Step: 93 ___________________________________________
loss: 20878.349609375


Step: 94 ___________________________________________
loss: 26181.978515625


Step: 95 ___________________________________________
loss: 27797.634765625


Step: 96 ___________________________________________
loss: 25078.033203125


Step: 97 ___________________________________________
loss: 18510.728515625


Step: 98 ___________________________________________
loss: 34419.02734375


Step: 99 ___________________________________________
loss: 23669.001953125


final_seed: 1533583633


In [46]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 2s 2s/step - loss: 13886951.0000
Epoch 2/10000
1/1 [==============================] - 0s 4ms/step - loss: 13317687.0000
Epoch 3/10000
1/1 [==============================] - 0s 4ms/step - loss: 4152762.7500
Epoch 4/10000
1/1 [==============================] - 0s 4ms/step - loss: 2947190.7500
Epoch 5/10000
1/1 [==============================] - 0s 4ms/step - loss: 1001373.3125
Epoch 6/10000
1/1 [==============================] - 0s 4ms/step - loss: 739763.0625
Epoch 7/10000
1/1 [==============================] - 0s 4ms/step - loss: 653764.2500
Epoch 8/10000
1/1 [==============================] - 0s 4ms/step - loss: 219373.5781
Epoch 9/10000
1/1 [==============================] - 0s 4ms/step - loss: 308395.3750
Epoch 10/10000
1/1 [==============================] - 0s 4ms/step - loss: 133119.4219
Epoch 11/10000
1/1 [==============================] - 0s 4ms/step - loss: 107942.9609
Epoch 12/10000
1/1 [==============================] - 0s 

1/1 [==============================] - 0s 4ms/step - loss: 104556.2500
Epoch 97/10000
1/1 [==============================] - 0s 4ms/step - loss: 107511.2422
Epoch 98/10000
1/1 [==============================] - 0s 4ms/step - loss: 127554.6875
Epoch 99/10000
1/1 [==============================] - 0s 4ms/step - loss: 97538.7734
Epoch 100/10000
1/1 [==============================] - 0s 4ms/step - loss: 52776.6211
Epoch 101/10000
1/1 [==============================] - 0s 4ms/step - loss: 122154.2891
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 110494.0078
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 53513.7305
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 104673.6875
Epoch 105/10000
1/1 [==============================] - 0s 4ms/step - loss: 172952.3750
Epoch 106/10000
1/1 [==============================] - 0s 5ms/step - loss: 146581.7969
Epoch 107/10000
1/1 [==============================] - 0s 6ms/ste

1/1 [==============================] - 0s 4ms/step - loss: 99868.1172
Epoch 191/10000
1/1 [==============================] - 0s 4ms/step - loss: 81675.1562
Epoch 192/10000
1/1 [==============================] - 0s 4ms/step - loss: 115232.2422
Epoch 193/10000
1/1 [==============================] - 0s 5ms/step - loss: 69529.7578
Epoch 194/10000
1/1 [==============================] - 0s 4ms/step - loss: 157427.7969
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 87380.4453
Epoch 196/10000
1/1 [==============================] - 0s 4ms/step - loss: 101043.6484
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 129991.3984
Epoch 198/10000
1/1 [==============================] - 0s 4ms/step - loss: 107470.7188
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 291748.6562
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 106551.9062
Epoch 201/10000
1/1 [==============================] - 0s 4ms/s

1/1 [==============================] - 0s 4ms/step - loss: 71445.5859
Epoch 285/10000
1/1 [==============================] - 0s 4ms/step - loss: 117583.8125
Epoch 286/10000
1/1 [==============================] - 0s 4ms/step - loss: 77198.3125
Epoch 287/10000
1/1 [==============================] - 0s 4ms/step - loss: 149991.9219
Epoch 288/10000
1/1 [==============================] - 0s 4ms/step - loss: 134021.0156
Epoch 289/10000
1/1 [==============================] - 0s 4ms/step - loss: 61788.6406
Epoch 290/10000
1/1 [==============================] - 0s 4ms/step - loss: 180764.1875
Epoch 291/10000
1/1 [==============================] - 0s 4ms/step - loss: 55623.9844
Epoch 292/10000
1/1 [==============================] - 0s 4ms/step - loss: 103916.6328
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 164837.0781
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 139026.6094
Epoch 295/10000
1/1 [==============================] - 0s 4ms/s

1/1 [==============================] - 0s 4ms/step - loss: 105380.2891
Epoch 379/10000
1/1 [==============================] - 0s 4ms/step - loss: 80433.2422
Epoch 380/10000
1/1 [==============================] - 0s 4ms/step - loss: 72435.4062
Epoch 381/10000
1/1 [==============================] - 0s 4ms/step - loss: 84581.1250
Epoch 382/10000
1/1 [==============================] - 0s 4ms/step - loss: 152467.8906
Epoch 383/10000
1/1 [==============================] - 0s 4ms/step - loss: 75967.0234
Epoch 384/10000
1/1 [==============================] - 0s 4ms/step - loss: 66641.1797
Epoch 385/10000
1/1 [==============================] - 0s 4ms/step - loss: 133189.7656
Epoch 386/10000
1/1 [==============================] - 0s 4ms/step - loss: 96012.3984
Epoch 387/10000
1/1 [==============================] - 0s 4ms/step - loss: 103821.7109
Epoch 388/10000
1/1 [==============================] - 0s 4ms/step - loss: 69552.5078
Epoch 389/10000
1/1 [==============================] - 0s 4ms/step

...

1/1 [==============================] - 0s 5ms/step - loss: 82291.9688
Epoch 849/10000
1/1 [==============================] - 0s 4ms/step - loss: 94688.4688
Epoch 850/10000
1/1 [==============================] - 0s 4ms/step - loss: 196754.1406
Epoch 851/10000
1/1 [==============================] - 0s 4ms/step - loss: 122905.8359
Epoch 852/10000
1/1 [==============================] - 0s 4ms/step - loss: 85554.2266
Epoch 853/10000
1/1 [==============================] - 0s 4ms/step - loss: 116959.3047
Epoch 854/10000
1/1 [==============================] - 0s 4ms/step - loss: 94188.4141
Epoch 855/10000
1/1 [==============================] - 0s 4ms/step - loss: 104283.9453
Epoch 856/10000
1/1 [==============================] - 0s 4ms/step - loss: 103843.1641
Epoch 857/10000
1/1 [==============================] - 0s 4ms/step - loss: 142344.3594
Epoch 858/10000
1/1 [==============================] - 0s 4ms/step - loss: 93043.4922
Epoch 859/10000
1/1 [==============================] - 0s 4ms/st

Epoch 943/10000
1/1 [==============================] - 0s 4ms/step - loss: 102840.3750
Epoch 944/10000
1/1 [==============================] - 0s 6ms/step - loss: 70017.1641
Epoch 945/10000
1/1 [==============================] - 0s 5ms/step - loss: 100702.7812
Epoch 946/10000
1/1 [==============================] - 0s 4ms/step - loss: 111448.6641
Epoch 947/10000
1/1 [==============================] - 0s 4ms/step - loss: 116574.2422
Epoch 948/10000
1/1 [==============================] - 0s 4ms/step - loss: 156271.2812
Epoch 949/10000
1/1 [==============================] - 0s 4ms/step - loss: 95443.1875
Epoch 950/10000
1/1 [==============================] - 0s 4ms/step - loss: 74605.6406
Epoch 951/10000
1/1 [==============================] - 0s 4ms/step - loss: 110337.1016
Epoch 952/10000
1/1 [==============================] - 0s 4ms/step - loss: 112978.5312
Epoch 953/10000
1/1 [==============================] - 0s 4ms/step - loss: 47037.5938
Epoch 954/10000
1/1 [==========================

1/1 [==============================] - 0s 6ms/step - loss: 123403.9609
Epoch 1038/10000
1/1 [==============================] - 0s 6ms/step - loss: 121485.6328
Epoch 1039/10000
1/1 [==============================] - 0s 5ms/step - loss: 98792.4141
Epoch 1040/10000
1/1 [==============================] - 0s 6ms/step - loss: 156749.3906
Epoch 1041/10000
1/1 [==============================] - 0s 5ms/step - loss: 53940.6250
Epoch 1042/10000
1/1 [==============================] - 0s 4ms/step - loss: 143855.2969
Epoch 1043/10000
1/1 [==============================] - 0s 4ms/step - loss: 107207.8359
Epoch 1044/10000
1/1 [==============================] - 0s 5ms/step - loss: 124351.5234
Epoch 1045/10000
1/1 [==============================] - 0s 5ms/step - loss: 112981.9297
Epoch 1046/10000
1/1 [==============================] - 0s 4ms/step - loss: 90813.8203
Epoch 1047/10000
1/1 [==============================] - 0s 5ms/step - loss: 92821.5000
Epoch 1048/10000
1/1 [==============================]

1/1 [==============================] - 0s 4ms/step - loss: 89568.3125
Epoch 1131/10000
1/1 [==============================] - 0s 4ms/step - loss: 110975.9453
Epoch 1132/10000
1/1 [==============================] - 0s 4ms/step - loss: 116665.3516
Epoch 1133/10000
1/1 [==============================] - 0s 5ms/step - loss: 75844.7578
Epoch 1134/10000
1/1 [==============================] - 0s 4ms/step - loss: 78291.0781
Epoch 1135/10000
1/1 [==============================] - 0s 4ms/step - loss: 135791.3125
Epoch 1136/10000
1/1 [==============================] - 0s 4ms/step - loss: 139005.1719
Epoch 1137/10000
1/1 [==============================] - 0s 4ms/step - loss: 93341.1562
Epoch 1138/10000
1/1 [==============================] - 0s 4ms/step - loss: 91054.5703
Epoch 1139/10000
1/1 [==============================] - 0s 6ms/step - loss: 78386.1641
Epoch 1140/10000
1/1 [==============================] - 0s 5ms/step - loss: 99706.2891
Epoch 1141/10000
1/1 [==============================] - 

1/1 [==============================] - 0s 4ms/step - loss: 94355.2578
Epoch 1224/10000
1/1 [==============================] - 0s 4ms/step - loss: 88280.8750
Epoch 1225/10000
1/1 [==============================] - 0s 4ms/step - loss: 149603.7812
Epoch 1226/10000
1/1 [==============================] - 0s 6ms/step - loss: 67956.6328
Epoch 1227/10000
1/1 [==============================] - 0s 4ms/step - loss: 98929.9141
Epoch 1228/10000
1/1 [==============================] - 0s 4ms/step - loss: 118052.0703
Epoch 1229/10000
1/1 [==============================] - 0s 4ms/step - loss: 124516.6641
Epoch 1230/10000
1/1 [==============================] - 0s 4ms/step - loss: 46430.6445
Epoch 1231/10000
1/1 [==============================] - 0s 4ms/step - loss: 108562.2734
Epoch 1232/10000
1/1 [==============================] - 0s 4ms/step - loss: 161242.4531
Epoch 1233/10000
1/1 [==============================] - 0s 4ms/step - loss: 98643.2109
Epoch 1234/10000
1/1 [==============================] -

In [47]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 575ms/step
[2829.682](test_target) - [2734.422](prediction) = 95.25988085937479


In [48]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.03366451808343651

In [49]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [50]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Rio de Janeiro - Desemprego,Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH Educacao,Rio de Janeiro - IDH,Rio de Janeiro - IDH Longevidade,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rio de Janeiro - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-0.435710,-1.280274,-1.213114,-1.217753,-1.331024,-0.871988,-1.413027,1.313806,0.083531,1.148205,...,1.308949,1.412346,1.330223,1.225197,-0.473348,-0.982967,0.870151,-0.11860,-1.399320,-1.282627
1,1.383023,0.119874,-0.022940,-0.013866,0.251650,-0.528230,0.656357,-1.110153,-1.264372,0.140893,...,-0.190804,-0.643250,-0.249322,-0.000905,-0.917430,-0.389046,0.530392,1.27973,0.522379,0.125415
2,-0.947314,1.160400,1.236054,1.231619,1.079374,1.400218,0.756670,-0.203653,1.180841,-1.289098,...,-1.118144,-0.769096,-1.080901,-1.224292,1.390778,1.372013,-1.400543,-1.16113,0.876940,1.157212


train_target:


,Rio De Janeiro - Consumo de Cimento (t)
0,3966.930
1,4199.911
2,4390.254


test_input:


...

,Rio de Janeiro - Desemprego,Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH Educacao,Rio de Janeiro - IDH,Rio de Janeiro - IDH Longevidade,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rio de Janeiro - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.841176,-0.224005,0.757994,-0.824847,-2.627309,0.3589,-0.879379,-0.909417,-1.121427,1.108405,...,-1.482779,-2.260447,-1.268801,-0.504495,0.492406,-0.267962,-0.64034,-0.480755,-1.410819,-1.69692


test_target:


,Rio De Janeiro - Consumo de Cimento (t)
12,2829.682


1/1 [==============================] - 1s 590ms/step
Error: 95.25988085937479




[4467.12353515625,
 4293.7900390625,
 4638.95556640625,
 4093.066162109375,
 3700.88330078125,
 2972.24853515625,
 2342.81640625,
 2492.742431640625,
 2608.443359375,
 2734.422119140625]

In [51]:
display(targets)
display(predictions)

[4733.6320000000005,
 4864.477,
 4454.468,
 3639.072,
 2811.648,
 2284.918,
 2391.212,
 2715.843,
 2935.607,
 2829.682]

[4467.12353515625,
 4293.7900390625,
 4638.95556640625,
 4093.066162109375,
 3700.88330078125,
 2972.24853515625,
 2342.81640625,
 2492.742431640625,
 2608.443359375,
 2734.422119140625]

In [52]:
mae = mean_absolute_error(predictions, targets)
mae

374.61626738281245

In [53]:
porcentage = mae/np.mean(targets)
porcentage

0.11129234882368187